In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
actions = [
    'come',
    'away',
    'spin'
]

data = np.concatenate([
    np.load('D:\Pycharm\PyCharm Community Edition 2020.3.3\HSU_BIT_Project\HSU_ProjectFile\dataset\seq_next_1640758559.npy'),
    np.load('D:\Pycharm\PyCharm Community Edition 2020.3.3\HSU_BIT_Project\HSU_ProjectFile\dataset\seq_prev_1640758559.npy'),
], axis=0)

data.shape

(1714, 30, 100)

In [3]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(1714, 30, 99)
(1714,)


In [4]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(1714, 3)

In [5]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(1542, 30, 99) (1542, 3)
(172, 30, 99) (172, 3)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                41984     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 44,163
Trainable params: 44,163
Non-trainable params: 0
_________________________________________________________________


In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=500,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/500
46/49 [===========================>..] - ETA: 0s - loss: 110.0078 - acc: 0.4694
Epoch 00001: val_acc improved from -inf to 0.52907, saving model to models\model.h5
49/49 [==============================] - 2s 17ms/step - loss: 107.9934 - acc: 0.4708 - val_loss: 55.3714 - val_acc: 0.5291 - lr: 0.0010
Epoch 2/500
45/49 [==========================>...] - ETA: 0s - loss: 47.2910 - acc: 0.5299
Epoch 00002: val_acc improved from 0.52907 to 0.56395, saving model to models\model.h5
49/49 [==============================] - 1s 13ms/step - loss: 46.3695 - acc: 0.5331 - val_loss: 33.2364 - val_acc: 0.5640 - lr: 0.0010
Epoch 3/500
48/49 [============================>.] - ETA: 0s - loss: 30.2476 - acc: 0.5358
Epoch 00003: val_acc did not improve from 0.56395
49/49 [==============================] - 1s 12ms/step - loss: 30.1868 - acc: 0.5357 - val_loss: 50.0357 - val_acc: 0.5465 - lr: 0.0010
Epoch 4/500
48/49 [============================>.] - ETA: 0s - loss: 60.8782 - acc: 0.5085
Epoch 00

49/49 [==============================] - 1s 11ms/step - loss: 8.8685 - acc: 0.6498 - val_loss: 15.5111 - val_acc: 0.5174 - lr: 0.0010
Epoch 30/500
47/49 [===========================>..] - ETA: 0s - loss: 10.2439 - acc: 0.6277
Epoch 00030: val_acc did not improve from 0.75581
49/49 [==============================] - 1s 11ms/step - loss: 10.0813 - acc: 0.6304 - val_loss: 27.7965 - val_acc: 0.5407 - lr: 0.0010
Epoch 31/500
45/49 [==========================>...] - ETA: 0s - loss: 8.8021 - acc: 0.6687
Epoch 00031: val_acc did not improve from 0.75581
49/49 [==============================] - 1s 11ms/step - loss: 9.3813 - acc: 0.6628 - val_loss: 12.5058 - val_acc: 0.6453 - lr: 0.0010
Epoch 32/500
46/49 [===========================>..] - ETA: 0s - loss: 7.9301 - acc: 0.6719
Epoch 00032: val_acc did not improve from 0.75581
49/49 [==============================] - 1s 11ms/step - loss: 7.8543 - acc: 0.6725 - val_loss: 9.0598 - val_acc: 0.6744 - lr: 0.0010
Epoch 33/500
46/49 [====================

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/nextprev')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))